# Torus one-point functions of the $O(n)$ model

In [1]:
using BootstrapVirasoro, BootstrapVirasoro.LoopModels, BenchmarkTools, BarnesDoubleGamma
println("Number of threads: $(Threads.nthreads())")

Number of threads: 64


In [89]:
const Sig = Channels{Rational}

Sig(σ) = Sig(σ, σ, σ)

# Spectrum of O(n) model
c = CC(β=sqrt(big"2") / (big"0.8" + big"0.2" * im))
diag_field = Field(c, P=(big"0.2" + big"0.1" * im)/sqrt(big"2"))
ndiag_indices = [(r, s) for r in 1//2:1//2:40 for s in -1+1//(2r):1//(2r):1 if r * s % 1 == 0]
fields = vcat([Field(c, r=r, s=s) for (r, s) in ndiag_indices], diag_field)

function precompute_blocks(indices, fields=nothing; c=CC(β=sqrt(big"2") / (big"0.8" + big"0.2" * im)),
    P=big"0.4" + big"0.1" * im, parity, precision=10)
    setprecision(BigFloat, floor(Int, 1.2 * precision), base=10)
    Δmax = floor(Int, 0.75 * precision)
    if fields === nothing
        diag_field = Field(c, P=P)
        ndiag_indices = [(r, s) for r in 1//2:1//2:40 for s in -1+1//(2r):1//(2r):1 if r * s % 1 == 0]
        fields = vcat([Field(c, r=r, s=s) for (r, s) in ndiag_indices], diag_field)
    end
    co = Correlation(Field(c, r=indices[1], s=indices[2]), Δmax)
    parity == 1 && (fields = filter(V -> V.diagonal || V.s >= 0, fields))
    parity == -1 && (fields = filter(V -> spin(V) != 0 && 0 < V.s && V.s < 1, fields))
    f = V -> IBlock(co, V, parity=parity)
    spec = ChanSpec(co, fields, f)
    return @channels spec
end

function compute_diagblocks(specs, Ps, chan)
    c = specs.s.corr.fields[1].c
    return [IBlock(specs.s.corr, chan, Field(c, P=P)) for P in Ps]
end

# Solve crossing symmetry for given signature
# Optionnally for many different diagonal blocks in the channel
function solve(
    specs, signature;
    int_r = nothing,
    rmax=3, show=true,
    Ps=nothing, diagchan=nothing,
    fix=nothing,
    rels=:stu
)
    specs = @channels filter(
        V -> V.diagonal && signature[chan] == 0 || !V.diagonal && V.r >= signature[chan],
        specs[chan]
    )
    if int_r !== nothing && int_r == true
        specs = @channels filter(V -> V.r isa Int, specs[chan])
    end
    if Ps === nothing
        res = solve_bootstrap(specs, fix=fix, rels=rels) # rels = :stu: structure constants equal in all channels.
        if show
            Base.show(stdout, res.str_cst.s, rmax=rmax) # print only s-channel, since others are equal.
        end
    else
        diagblocks = compute_diagblocks(specs, Ps, diagchan)
        res = solve_bootstrap_manyP(specs, diagblocks)
    end
    return res
end;

# Count dimensions of spaces of solutions

| r                       | 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 | 10 | 11 | 12 | 13 | 14 |
|-------------------------|---|---|---|---|---|---|---|---|---|---|----|----|----|----|----|
|number of solutions      | 1 | 1 | 2 | 3 | 4 | 5 | 8 | 9 |12 |15 | 18 | 21 | 26 | 29 | 34 |
|number of even solutions | 1 | 1 | 2 | 3 | 4 | 5 | 7 | 8 |10 |12 | 14 | 16 | 19 | 20 |    |
|number of odd  solutions | 0 | 0 | 0 | 0 | 0 | 0 | 1 | 1 |2  |3  | 4  | 5  | 7  | 9  |    |

In [3]:
fixed_constants_order = Dict(
    0 => [(:s, diag_field, 1), [(:s, V, 0) for V in fields[1:end-1]]...,],
    1 => [(:s, diag_field, 1), [(:s, V, 0) for V in fields[1:end-1] if V.s >= 0]...,],
   -1 => [[:s, V, 0] for V in fields[1:end-1] if spin(V) != 0 && 0 < V.s && V.s < 1],
)
fixed_constants_order[-1][1][3] = 1

nb_comb_maps = Dict(
    0 => [1, 2, 3, 4, 5, 8, 9, 12, 15, 18, 21, 26, 29, 34, 39], # number of solutions
    1 => [1, 2, 3, 4, 5, 7, 8, 10, 12, 14, 16, 19, 20], # number of parity-even solutions
   -1 => [0, 0, 0, 0, 0, 1, 1, 2, 3, 4, 5, 7, 9,], # number of parity-odd solutions
)

precisions = [40, 40, 40, 40, 40, 40, 60, 65, 75, 85, 90, 95, 100, 130]

function check_dimension(r, s; parity=0, expected_dimension=nothing, not_fixed = [], precision=nothing, blocks=nothing)
    printstyled("=====\nr = $r\n=====\n", bold=true)
    (expected_dimension === nothing) && (expected_dimension = nb_comb_maps[parity][r])
    (precision === nothing) && (precision = precisions[r])
    (blocks === nothing) && (blocks = precompute_blocks((r, s), fields, parity=parity, precision=precisions[r]))
    fixed = [f for f in fixed_constants_order[parity][1:expected_dimension + length(not_fixed)] if !(f in not_fixed)]
    println("With $(length(fixed)) constants fixed:")
    solve(blocks, Sig(0, 0, 0), fix=fixed, rmax=(r + 3) // 3)
    if length(fixed) > 1
        println("With $(length(fixed)-1) constant fixed:")
        solve(blocks, Sig(0, 0, 0), fix=fixed[1:end-1], rmax=(r + 3) // 3)
    end
end;

## Parity unfixed

### $r \in \{1, \dots, 8\} \cup \{ 10\}$

In [59]:
@time for r in [1:8..., 10]
    check_dimension(r, 0)
end

=====
r = 1
=====
With 1 constants fixed:
┌────────────────────┬────────────────────┬───────────┐
│ Field              │ Structure constant │ Rel. err. │
├────────────────────┼────────────────────┼───────────┤
│ P = (0.11+0.057im) │ 1.0+0.0im          │ 0         │
│ (1//2, 0)          │ 6.2154-4.35819im   │ 1.5e-44   │
│ (1, 0)             │ 5.84666-10.1272im  │ 8.2e-43   │
│ (1, 1)             │ -4.14065-1.37768im │ 5.2e-43   │
└────────────────────┴────────────────────┴───────────┘
=====
r = 2
=====
With 2 constants fixed:
┌────────────────────┬────────────────────┬───────────┐
│ Field              │ Structure constant │ Rel. err. │
├────────────────────┼────────────────────┼───────────┤
│ P = (0.11+0.057im) │ 1.0+0.0im          │ 0         │
│ (1//2, 0)          │ 0.0+0.0im          │ 0         │
│ (1, 0)             │ -934.438+210.945im │ 9.4e-45   │
│ (1, 1)             │ -597.189+296.177im │ 1.9e-44   │
│ (3//2, 0)          │ 1538.45+4136.88im  │ 2.7e-41   │
│ (3//2, -2//3)     

### $r = 9$

In [27]:
check_dimension(9, 0, not_fixed=fixed_constants_order[0][14:15]);

=====
r = 9
=====
With 15 constants fixed:
┌────────────────────┬────────────────────────────┬───────────┐
│ Field              │ Structure constant         │ Rel. err. │
├────────────────────┼────────────────────────────┼───────────┤
│ P = (0.11+0.057im) │ 1.0+0.0im                  │ 0         │
│ (1//2, 0)          │ 0.0+0.0im                  │ 0         │
│ (1, 0)             │ 0.0+0.0im                  │ 0         │
│ (1, 1)             │ 0.0+0.0im                  │ 0         │
│ (3//2, 0)          │ 0.0+0.0im                  │ 0         │
│ (3//2, -2//3)      │ 0.0+0.0im                  │ 0         │
│ (3//2, 2//3)       │ 0.0+0.0im                  │ 0         │
│ (2, 0)             │ 0.0+0.0im                  │ 0         │
│ (2, 1//2)          │ 0.0+0.0im                  │ 0         │
│ (2, -1//2)         │ 0.0+0.0im                  │ 0         │
│ (2, 1)             │ 0.0+0.0im                  │ 0         │
│ (5//2, 0)          │ 1.19433e+25-1.32957e+25im  │ 1.9e-41  

Note that with the first 14 constants fixed, the two remaining solutions both have the same $D_{(\frac72, s)}$ and $D_{(\frac52, s)}$, but not the same $D_{(3, s)}$.

### $r=11$

In [34]:
r = 11
blocks = precompute_blocks((r, 0), fields, parity=0, precision=precisions[r]);

In [35]:
check_dimension(11, 0, blocks=blocks);

=====
r = 11
=====
With 21 constants fixed:
┌────────────────────┬────────────────────────────┬───────────┐
│ Field              │ Structure constant         │ Rel. err. │
├────────────────────┼────────────────────────────┼───────────┤
│ P = (0.11+0.057im) │ 1.0+0.0im                  │ 0         │
│ (1//2, 0)          │ 0.0+0.0im                  │ 0         │
│ (1, 0)             │ 0.0+0.0im                  │ 0         │
│ (1, 1)             │ 0.0+0.0im                  │ 0         │
│ (3//2, 0)          │ 0.0+0.0im                  │ 0         │
│ (3//2, -2//3)      │ 0.0+0.0im                  │ 0         │
│ (3//2, 2//3)       │ 0.0+0.0im                  │ 0         │
│ (2, 0)             │ 0.0+0.0im                  │ 0         │
│ (2, 1//2)          │ 0.0+0.0im                  │ 0         │
│ (2, -1//2)         │ 0.0+0.0im                  │ 0         │
│ (2, 1)             │ 0.0+0.0im                  │ 0         │
│ (5//2, 0)          │ 0.0+0.0im                  │ 0       

If we fix the first 20 constants, $D_{(\frac32, \frac23)} = 0$ in all the remaining solutions. So we need to set something else to zero. The following works (the two red lines are zero):

In [36]:
check_dimension(11, 0, blocks=blocks, not_fixed=fixed_constants_order[0][19:20], expected_dimension=21);

=====
r = 11
=====
With 21 constants fixed:
┌────────────────────┬────────────────────────────┬───────────┐
│ Field              │ Structure constant         │ Rel. err. │
├────────────────────┼────────────────────────────┼───────────┤
│ P = (0.11+0.057im) │ 1.0+0.0im                  │ 0         │
│ (1//2, 0)          │ 0.0+0.0im                  │ 0         │
│ (1, 0)             │ 0.0+0.0im                  │ 0         │
│ (1, 1)             │ 0.0+0.0im                  │ 0         │
│ (3//2, 0)          │ 0.0+0.0im                  │ 0         │
│ (3//2, -2//3)      │ 0.0+0.0im                  │ 0         │
│ (3//2, 2//3)       │ 0.0+0.0im                  │ 0         │
│ (2, 0)             │ 0.0+0.0im                  │ 0         │
│ (2, 1//2)          │ 0.0+0.0im                  │ 0         │
│ (2, -1//2)         │ 0.0+0.0im                  │ 0         │
│ (2, 1)             │ 0.0+0.0im                  │ 0         │
│ (5//2, 0)          │ 0.0+0.0im                  │ 0       

### $r=12$

In [52]:
check_dimension(12, 0, not_fixed=fixed_constants_order[0][26:29]);

=====
r = 12
=====
With 26 constants fixed:
┌────────────────────┬────────────────────────────┬───────────┐
│ Field              │ Structure constant         │ Rel. err. │
├────────────────────┼────────────────────────────┼───────────┤
│ P = (0.11+0.057im) │ 1.0+0.0im                  │ 0         │
│ (1//2, 0)          │ 0.0+0.0im                  │ 0         │
│ (1, 0)             │ 0.0+0.0im                  │ 0         │
│ (1, 1)             │ 0.0+0.0im                  │ 0         │
│ (3//2, 0)          │ 0.0+0.0im                  │ 0         │
│ (3//2, -2//3)      │ 0.0+0.0im                  │ 0         │
│ (3//2, 2//3)       │ 0.0+0.0im                  │ 0         │
│ (2, 0)             │ 0.0+0.0im                  │ 0         │
│ (2, 1//2)          │ 0.0+0.0im                  │ 0         │
│ (2, -1//2)         │ 0.0+0.0im                  │ 0         │
│ (2, 1)             │ 0.0+0.0im                  │ 0         │
│ (5//2, 0)          │ 0.0+0.0im                  │ 0       

### $r=13$

In [58]:
check_dimension(13, 0);

=====
r = 13
=====
With 29 constants fixed:
┌────────────────────┬──────────────────────────────┬───────────┐
│ Field              │ Structure constant           │ Rel. err. │
├────────────────────┼──────────────────────────────┼───────────┤
│ P = (0.11+0.057im) │ 1.0+0.0im                    │ 0         │
│ (1//2, 0)          │ 0.0+0.0im                    │ 0         │
│ (1, 0)             │ 0.0+0.0im                    │ 0         │
│ (1, 1)             │ 0.0+0.0im                    │ 0         │
│ (3//2, 0)          │ 0.0+0.0im                    │ 0         │
│ (3//2, -2//3)      │ 0.0+0.0im                    │ 0         │
│ (3//2, 2//3)       │ 0.0+0.0im                    │ 0         │
│ (2, 0)             │ 0.0+0.0im                    │ 0         │
│ (2, 1//2)          │ 0.0+0.0im                    │ 0         │
│ (2, -1//2)         │ 0.0+0.0im                    │ 0         │
│ (2, 1)             │ 0.0+0.0im                    │ 0         │
│ (5//2, 0)          │ 0.0+0.0im

In [74]:
fixed_constants_order[0][26:29]

4-element Vector{Tuple{Symbol, Field{Complex{BigFloat}}, Int64}}:
 (:s, (7//2, 0), 0)
 (:s, (7//2, 2//7), 0)
 (:s, (7//2, 4//7), 0)
 (:s, (7//2, 6//7), 0)

In [55]:
check_dimension(13, 0, not_fixed=fixed_constants_order[0][26:29]);

=====
r = 13
=====
With 29 constants fixed:
┌────────────────────┬────────────────────────────┬───────────┐
│ Field              │ Structure constant         │ Rel. err. │
├────────────────────┼────────────────────────────┼───────────┤
│ P = (0.11+0.057im) │ 1.0+0.0im                  │ 0         │
│ (1//2, 0)          │ 0.0+0.0im                  │ 0         │
│ (1, 0)             │ 0.0+0.0im                  │ 0         │
│ (1, 1)             │ 0.0+0.0im                  │ 0         │
│ (3//2, 0)          │ 0.0+0.0im                  │ 0         │
│ (3//2, -2//3)      │ 0.0+0.0im                  │ 0         │
│ (3//2, 2//3)       │ 0.0+0.0im                  │ 0         │
│ (2, 0)             │ 0.0+0.0im                  │ 0         │
│ (2, 1//2)          │ 0.0+0.0im                  │ 0         │
│ (2, -1//2)         │ 0.0+0.0im                  │ 0         │
│ (2, 1)             │ 0.0+0.0im                  │ 0         │
│ (5//2, 0)          │ 0.0+0.0im                  │ 0       

### $r=14$

In [57]:
check_dimension(14, 0, not_fixed=fixed_constants_order[0][26:32]);

=====
r = 14
=====
With 34 constants fixed:
┌────────────────────┬──────────────────────────────┬───────────┐
│ Field              │ Structure constant           │ Rel. err. │
├────────────────────┼──────────────────────────────┼───────────┤
│ P = (0.11+0.057im) │ 1.0+0.0im                    │ 0         │
│ (1//2, 0)          │ 0.0+0.0im                    │ 0         │
│ (1, 0)             │ 0.0+0.0im                    │ 0         │
│ (1, 1)             │ 0.0+0.0im                    │ 0         │
│ (3//2, 0)          │ 0.0+0.0im                    │ 0         │
│ (3//2, -2//3)      │ 0.0+0.0im                    │ 0         │
│ (3//2, 2//3)       │ 0.0+0.0im                    │ 0         │
│ (2, 0)             │ 0.0+0.0im                    │ 0         │
│ (2, 1//2)          │ 0.0+0.0im                    │ 0         │
│ (2, -1//2)         │ 0.0+0.0im                    │ 0         │
│ (2, 1)             │ 0.0+0.0im                    │ 0         │
│ (5//2, 0)          │ 0.0+0.0im

## Parity-even solutions

### $r \in \{1, \dots, 8\} \cup \{10\}$

In [ ]:
for r = [1:8..., 10]
    check_dimension(r, 0, parity=1)
end

=====
r = 1
=====
With 1 constants fixed:
┌────────────────────┬────────────────────┬───────────┐
│ Field              │ Structure constant │ Rel. err. │
├────────────────────┼────────────────────┼───────────┤
│ P = (0.11+0.057im) │ 1.0+0.0im          │ 0         │
│ (1//2, 0)          │ 6.2154-4.35819im   │ 8.1e-46   │
│ (1, 0)             │ 5.84666-10.1272im  │ 4.1e-44   │
│ (1, 1)             │ -4.14065-1.37768im │ 6.3e-44   │
└────────────────────┴────────────────────┴───────────┘
=====
r = 2
=====
With 2 constants fixed:
┌────────────────────┬────────────────────┬───────────┐
│ Field              │ Structure constant │ Rel. err. │
├────────────────────┼────────────────────┼───────────┤
│ P = (0.11+0.057im) │ 1.0+0.0im          │ 0         │
│ (1//2, 0)          │ 0.0+0.0im          │ 0         │
│ (1, 0)             │ -934.438+210.945im │ 2.8e-45   │
│ (1, 1)             │ -597.189+296.177im │ 1.6e-44   │
│ (3//2, 0)          │ 1538.45+4136.88im  │ 1e-41     │
│ (3//2, 2//3)      

### $r=9$

In [32]:
check_dimension(9, 0, parity=1);

=====
r = 9
=====
With 12 constants fixed:
┌────────────────────┬────────────────────────────┬───────────┐
│ Field              │ Structure constant         │ Rel. err. │
├────────────────────┼────────────────────────────┼───────────┤
│ P = (0.11+0.057im) │ 1.0+0.0im                  │ 0         │
│ (1//2, 0)          │ 0.0+0.0im                  │ 0         │
│ (1, 0)             │ 0.0+0.0im                  │ 0         │
│ (1, 1)             │ 0.0+0.0im                  │ 0         │
│ (3//2, 0)          │ 0.0+0.0im                  │ 0         │
│ (3//2, 2//3)       │ 0.0+0.0im                  │ 0         │
│ (2, 0)             │ 0.0+0.0im                  │ 0         │
│ (2, 1//2)          │ 0.0+0.0im                  │ 0         │
│ (2, 1)             │ 0.0+0.0im                  │ 0         │
│ (5//2, 0)          │ 0.0+0.0im                  │ 0         │
│ (5//2, 2//5)       │ 0.0+0.0im                  │ 0         │
│ (5//2, 4//5)       │ 0.0+0.0im                  │ 0        

Again we observe a relation between the $D_{(\frac52, s)}$ once we fix the $D_{(r\leq 2, s)}$. We get a unique solution if we fix one of the $D_{(3, s)}$:

In [37]:
check_dimension(9, 0, parity=1, expected_dimension = 12, not_fixed=fixed_constants_order[1][10:10]);

=====
r = 9
=====
With 12 constants fixed:
┌────────────────────┬────────────────────────────┬───────────┐
│ Field              │ Structure constant         │ Rel. err. │
├────────────────────┼────────────────────────────┼───────────┤
│ P = (0.11+0.057im) │ 1.0+0.0im                  │ 0         │
│ (1//2, 0)          │ 0.0+0.0im                  │ 0         │
│ (1, 0)             │ 0.0+0.0im                  │ 0         │
│ (1, 1)             │ 0.0+0.0im                  │ 0         │
│ (3//2, 0)          │ 0.0+0.0im                  │ 0         │
│ (3//2, 2//3)       │ 0.0+0.0im                  │ 0         │
│ (2, 0)             │ 0.0+0.0im                  │ 0         │
│ (2, 1//2)          │ 0.0+0.0im                  │ 0         │
│ (2, 1)             │ 0.0+0.0im                  │ 0         │
│ (5//2, 0)          │ 1.19433e+25-1.32957e+25im  │ 1e-40     │
│ (5//2, 2//5)       │ 0.0+0.0im                  │ 0         │
│ (5//2, 4//5)       │ 0.0+0.0im                  │ 0        

### $r=11$

In [ ]:
r = 11
check_dimension(r, 0, parity=1, not_fixed=fixed_constants_order[1][15:16]);

=====
r = 11
=====
With 16 constants fixed:
┌────────────────────┬────────────────────────────┬───────────┐
│ Field              │ Structure constant         │ Rel. err. │
├────────────────────┼────────────────────────────┼───────────┤
│ P = (0.11+0.057im) │ 1.0+0.0im                  │ 0         │
│ (1//2, 0)          │ 0.0+0.0im                  │ 0         │
│ (1, 0)             │ 0.0+0.0im                  │ 0         │
│ (1, 1)             │ 0.0+0.0im                  │ 0         │
│ (3//2, 0)          │ 0.0+0.0im                  │ 0         │
│ (3//2, 2//3)       │ 0.0+0.0im                  │ 0         │
│ (2, 0)             │ 0.0+0.0im                  │ 0         │
│ (2, 1//2)          │ 0.0+0.0im                  │ 0         │
│ (2, 1)             │ 0.0+0.0im                  │ 0         │
│ (5//2, 0)          │ 0.0+0.0im                  │ 0         │
│ (5//2, 2//5)       │ 0.0+0.0im                  │ 0         │
│ (5//2, 4//5)       │ 0.0+0.0im                  │ 0       

### $r=12$

In [68]:
r = 12
check_dimension(r, 0, parity=1, not_fixed=fixed_constants_order[1][19:20]);

=====
r = 12
=====
With 19 constants fixed:
┌────────────────────┬────────────────────────────┬───────────┐
│ Field              │ Structure constant         │ Rel. err. │
├────────────────────┼────────────────────────────┼───────────┤
│ P = (0.11+0.057im) │ 1.0+0.0im                  │ 0         │
│ (1//2, 0)          │ 0.0+0.0im                  │ 0         │
│ (1, 0)             │ 0.0+0.0im                  │ 0         │
│ (1, 1)             │ 0.0+0.0im                  │ 0         │
│ (3//2, 0)          │ 0.0+0.0im                  │ 0         │
│ (3//2, 2//3)       │ 0.0+0.0im                  │ 0         │
│ (2, 0)             │ 0.0+0.0im                  │ 0         │
│ (2, 1//2)          │ 0.0+0.0im                  │ 0         │
│ (2, 1)             │ 0.0+0.0im                  │ 0         │
│ (5//2, 0)          │ 0.0+0.0im                  │ 0         │
│ (5//2, 2//5)       │ 0.0+0.0im                  │ 0         │
│ (5//2, 4//5)       │ 0.0+0.0im                  │ 0       

### $r=13$

In [79]:
r = 13
check_dimension(r, 0, parity=1, not_fixed=fixed_constants_order[1][17:22]);

=====
r = 13
=====
With 20 constants fixed:
┌────────────────────┬────────────────────────────┬───────────┐
│ Field              │ Structure constant         │ Rel. err. │
├────────────────────┼────────────────────────────┼───────────┤
│ P = (0.11+0.057im) │ 1.0+0.0im                  │ 0         │
│ (1//2, 0)          │ 0.0+0.0im                  │ 0         │
│ (1, 0)             │ 0.0+0.0im                  │ 0         │
│ (1, 1)             │ 0.0+0.0im                  │ 0         │
│ (3//2, 0)          │ 0.0+0.0im                  │ 0         │
│ (3//2, 2//3)       │ 0.0+0.0im                  │ 0         │
│ (2, 0)             │ 0.0+0.0im                  │ 0         │
│ (2, 1//2)          │ 0.0+0.0im                  │ 0         │
│ (2, 1)             │ 0.0+0.0im                  │ 0         │
│ (5//2, 0)          │ 0.0+0.0im                  │ 0         │
│ (5//2, 2//5)       │ 0.0+0.0im                  │ 0         │
│ (5//2, 4//5)       │ 0.0+0.0im                  │ 0       

## Parity-odd solutions

In [34]:
@time for r in 6:12
    check_dimension(r, 0, parity=-1)
end

=====
r = 6
=====
With 1 constants fixed:
┌──────────────┬────────────────────────────┬───────────┐
│ Field        │ Structure constant         │ Rel. err. │
├──────────────┼────────────────────────────┼───────────┤
│ (3//2, 2//3) │ 1.0+0.0im                  │ 0         │
│ (2, 1//2)    │ -353400.0+378860.0im       │ 3.9e-29   │
│ (5//2, 2//5) │ -4.93811e+10-2.13787e+11im │ 3e-27     │
│ (5//2, 4//5) │ 8.97642e+10+3.08534e+11im  │ 5.1e-28   │
│ (3, 1//3)    │ -1.50455e+13-4.45181e+12im │ 7.5e-21   │
│ (3, 2//3)    │ -1.22313e+13-1.97343e+12im │ 9.4e-21   │
└──────────────┴────────────────────────────┴───────────┘
=====
r = 7
=====
With 1 constants fixed:
┌──────────────┬────────────────────────────┬───────────┐
│ Field        │ Structure constant         │ Rel. err. │
├──────────────┼────────────────────────────┼───────────┤
│ (3//2, 2//3) │ 1.0+0.0im                  │ 0         │
│ (2, 1//2)    │ 0                          │ 3         │
│ (5//2, 2//5) │ 1.14734e+13-9.00323e+12im  │ 

In [32]:
check_dimension(13, 0, parity=-1, not_fixed=fixed_constants_order[-1][9:10]);

=====
r = 13
=====
With 9 constants fixed:
┌──────────────┬────────────────────────────┬───────────┐
│ Field        │ Structure constant         │ Rel. err. │
├──────────────┼────────────────────────────┼───────────┤
│ (3//2, 2//3) │ 1.0+0.0im                  │ 0         │
│ (2, 1//2)    │ 0.0+0.0im                  │ 0         │
│ (5//2, 2//5) │ 0.0+0.0im                  │ 0         │
│ (5//2, 4//5) │ 0.0+0.0im                  │ 0         │
│ (3, 1//3)    │ 0.0+0.0im                  │ 0         │
│ (3, 2//3)    │ 0.0+0.0im                  │ 0         │
│ (7//2, 2//7) │ 0.0+0.0im                  │ 0         │
│ (7//2, 4//7) │ 0.0+0.0im                  │ 0         │
│ (7//2, 6//7) │ 3.06632e+35-3.38827e+34im  │ 3e-12     │
│ (4, 1//4)    │ -3.8057e+47-3.98064e+46im  │ 1.4e-11   │
│ (4, 1//2)    │ 0.0+0.0im                  │ 0         │
│ (4, 3//4)    │ 4.08535e+47+5.92417e+46im  │ 2.3e-11   │
│ (9//2, 2//9) │ 4.1809e+63+3.24559e+63im   │ 2.7e-13   │
│ (9//2, 4//9) │ 6.42907e+63+

# Solutions with top signature

In [82]:
sigs = [Sig(i // 2) for i in 0:9]
precisions = [40, 40, 40, 40, 40, 40, 60, 80, 100, 130]
for r in 1:14
    if r == 1 || !((r-1) % 3 == 0)
        printstyled("=============\nr = $r\n=============\n", bold=true, color=:blue)
        ind = r - (r-2) ÷ 3
        sig = sigs[ind]
        blocks = precompute_blocks((r, 0), precision=precisions[ind], parity=1)
        solve(blocks, sig, rmax=r+2)
    end
end

r = 1
┌─────────────────┬──────────────────────┬───────────┐
│ Field           │ Structure constant   │ Rel. err. │
├─────────────────┼──────────────────────┼───────────┤
│ P = (0.4+0.1im) │ 1.0+0.0im            │ 0         │
│ (1//2, 0)       │ -1.85579+1.04513im   │ 2.6e-45   │
│ (1, 0)          │ -0.691991+0.509808im │ 5.2e-43   │
│ (1, 1)          │ 2.22426-1.63557im    │ 9.7e-44   │
│ (3//2, 0)       │ -1.75498+1.34764im   │ 5.1e-40   │
│ (3//2, 2//3)    │ 2.38055-1.76623im    │ 4.1e-40   │
│ (2, 0)          │ -3.11385+2.42417im   │ 3.7e-35   │
│ (2, 1//2)       │ -1.9265+1.44319im    │ 7.9e-35   │
│ (2, 1)          │ 2.06139-1.66833im    │ 7.7e-35   │
│ (5//2, 0)       │ -5.24947+4.26608im   │ 1.3e-28   │
│ (5//2, 2//5)    │ 0.944912-0.782701im  │ 7.5e-28   │
│ (5//2, 4//5)    │ 0.940418-0.783574im  │ 8.1e-28   │
│ (3, 0)          │ 1.06836-0.755555im   │ 9.4e-20   │
│ (3, 1//3)       │ -1.44922+1.20726im   │ 6.5e-20   │
│ (3, 2//3)       │ 2.67629-2.19646im    │ 3.5e-20   │
│ (3

In [70]:
for r in 1:8
    if r == 1 || !((r-1) % 3 == 0)
        printstyled("=============\nr = $r\n=============\n", bold=true, color=:blue)
        ind = r - (r-2) ÷ 3
        sig = sigs[ind]
        blocks = precompute_blocks((r, 2//r), precision=precisions[ind], parity=0)
        solve(blocks, sig, rmax=r+2)
    end
end

r = 1
┌─────────────────┬─────────────────────┬───────────┐
│ Field           │ Structure constant  │ Rel. err. │
├─────────────────┼─────────────────────┼───────────┤
│ P = (0.4+0.1im) │ 1.0+0.0im           │ 0         │
│ (1//2, 0)       │ 1.47438-0.711929im  │ 6.1e-46   │
│ (1, 0)          │ 1.08687-0.292643im  │ 4.1e-43   │
│ (1, 1)          │ -6.57613-0.31829im  │ 5.6e-44   │
│ (3//2, 0)       │ 3.75454-0.517584im  │ 6.2e-40   │
│ (3//2, -2//3)   │ -3.73493+1.06769im  │ 1.8e-40   │
│ (3//2, 2//3)    │ -6.6259-0.123444im  │ 2.6e-40   │
│ (2, 0)          │ 8.09886-0.371676im  │ 6e-35     │
│ (2, 1//2)       │ 5.73741+0.296272im  │ 6.6e-35   │
│ (2, -1//2)      │ 4.22831-0.474283im  │ 6.9e-35   │
│ (2, 1)          │ -7.32119-0.660551im │ 3e-35     │
│ (5//2, 0)       │ 15.9572+0.116328im  │ 1.3e-28   │
│ (5//2, -2//5)   │ -2.6252+0.144438im  │ 1.1e-27   │
│ (5//2, 2//5)    │ -3.18228-0.154964im │ 3.4e-28   │
│ (5//2, -4//5)   │ -2.36658+0.269965im │ 1.2e-27   │
│ (5//2, 4//5)    │ -3

In [85]:
blocks = precompute_blocks((2, 0), parity=0, precision=60)
solve(blocks, Sig(1//2));

┌───────────────┬────────────────────┬───────────┐
│ Field         │ Structure constant │ Rel. err. │
├───────────────┼────────────────────┼───────────┤
│ (1//2, 0)     │ 1.0+0.0im          │ 0         │
│ (1, 0)        │ -34.9208-45.5559im │ 5.3e-69   │
│ (1, 1)        │ 31.0888+25.0735im  │ 7e-69     │
│ (3//2, 0)     │ -589.454+35.4061im │ 4.8e-66   │
│ (3//2, -2//3) │ 277.297-26.5527im  │ 3.5e-66   │
│ (3//2, 2//3)  │ 277.297-26.5527im  │ 1.4e-66   │
│ (2, 0)        │ -1371.03+1178.54im │ 4.5e-61   │
│ (2, 1//2)     │ 0                  │ 1.6       │
│ (2, -1//2)    │ 0                  │ 1.2       │
│ (2, 1)        │ 1230.46-1153.33im  │ 4e-61     │
│ (5//2, 0)     │ -3042.25+7935.63im │ 1.9e-54   │
│ (5//2, -2//5) │ 746.95-1973.78im   │ 1.1e-53   │
│ (5//2, 2//5)  │ 746.95-1973.78im   │ 8e-55     │
│ (5//2, -4//5) │ 707.099-1941.71im  │ 1.1e-53   │
│ (5//2, 4//5)  │ 707.099-1941.71im  │ 8.5e-54   │
│ (3, 0)        │ 289.564+8488.92im  │ 4.8e-46   │
│ (3, -1//3)    │ 154.671+4232.

In [80]:
blocks = precompute_blocks((2, 1), parity=0, precision=60)
solve(blocks, Sig(1//2));

┌───────────────┬────────────────────────────┬───────────┐
│ Field         │ Structure constant         │ Rel. err. │
├───────────────┼────────────────────────────┼───────────┤
│ (1//2, 0)     │ 1.0+0.0im                  │ 0         │
│ (1, 0)        │ 413.74-62.3139im           │ 0.15      │
│ (1, 1)        │ 682.086+303.751im          │ 0.31      │
│ (3//2, 0)     │ -1.99335e+06-3.44189e+06im │ 2.8       │
│ (3//2, -2//3) │ -479164.0-2.40523e+06im    │ 1.1       │
│ (3//2, 2//3)  │ -2.19299e+06-2.06598e+06im │ 0.59      │
│ (2, 0)        │ 1.61308e+12+4.17684e+11im  │ 2.1       │
│ (2, 1//2)     │ 1.21746e+12-6.12886e+11im  │ 2.2       │
│ (2, -1//2)    │ 1.51591e+12+1.11765e+11im  │ 1.1       │
│ (2, 1)        │ 1.31143e+12-6.96724e+11im  │ 0.78      │
│ (5//2, 0)     │ 1.78656e+19+4.04732e+17im  │ 2.1       │
│ (5//2, -2//5) │ 1.70553e+19+2.76119e+18im  │ 1.4       │
│ (5//2, 2//5)  │ 2.18061e+19-8.41759e+18im  │ 3.1       │
│ (5//2, -4//5) │ 1.68967e+19-4.57639e+18im  │ 1.1      

In [79]:
blocks = precompute_blocks((2, 2), parity=0, precision=60)
solve(blocks, Sig(1//2));

┌───────────────┬────────────────────┬───────────┐
│ Field         │ Structure constant │ Rel. err. │
├───────────────┼────────────────────┼───────────┤
│ (1//2, 0)     │ 1.0+0.0im          │ 0         │
│ (1, 0)        │ 31.0638+27.001im   │ 1.9e-69   │
│ (1, 1)        │ -27.2602-139.431im │ 5.5e-70   │
│ (3//2, 0)     │ 685.966+14.0057im  │ 1.7e-66   │
│ (3//2, -2//3) │ -164.578+66.3112im │ 1.7e-66   │
│ (3//2, 2//3)  │ -562.381-177.725im │ 2.8e-66   │
│ (2, 0)        │ 2340.0-1278.06im   │ 1.5e-61   │
│ (2, 1//2)     │ 0                  │ 2.1       │
│ (2, -1//2)    │ 0                  │ 0.67      │
│ (2, 1)        │ -4618.68+953.003im │ 3.1e-61   │
│ (5//2, 0)     │ 9072.29-11611.6im  │ 4.8e-55   │
│ (5//2, -2//5) │ -1563.37+2554.54im │ 3.8e-54   │
│ (5//2, 2//5)  │ -3097.12+3222.07im │ 3.4e-54   │
│ (5//2, -4//5) │ -983.146+2188.89im │ 5.5e-54   │
│ (5//2, 4//5)  │ -4044.45+3493.48im │ 5.7e-54   │
│ (3, 0)        │ 5393.55-15774.0im  │ 3.3e-46   │
│ (3, -1//3)    │ 1811.08-7016.

The solution disappears for $s=1$, for some reason. This is not understood.

For $r=3$, the top-signature solution has only integer $r$ in the channel.

In [103]:
blocks = precompute_blocks((6, 2), parity=0, precision=60)
solve(blocks, Sig(2), int_r=true, rmax=4);

┌────────────┬────────────────────────────┬───────────┐
│ Field      │ Structure constant         │ Rel. err. │
├────────────┼────────────────────────────┼───────────┤
│ (2, 0)     │ 1.0+0.0im                  │ 0         │
│ (2, -1//2) │ -0.616057+0.158402im       │ 1e-44     │
│ (2, 1//2)  │ -1.58137-0.425172im        │ 1.3e-44   │
│ (2, 1)     │ 2.39225+1.44465im          │ 2.9e-45   │
│ (3, 0)     │ 297556.0-495551.0im        │ 5.8e-35   │
│ (3, 1//3)  │ -454166.0+339646.0im       │ 1.3e-34   │
│ (3, -1//3) │ 1320.25+127550.0im         │ 1.5e-34   │
│ (3, 2//3)  │ -886003.0+341079.0im       │ 1.3e-34   │
│ (3, -2//3) │ 36883.7+37226.9im          │ 1.3e-34   │
│ (3, 1)     │ 2.67028e+06-487116.0im     │ 4.9e-35   │
│ (4, 0)     │ -1.19727e+18+2.39128e+17im │ 1.2e-25   │
│ (4, 1//4)  │ 1.60035e+18-8.1815e+16im   │ 1.2e-25   │
│ (4, -1//4) │ 8.44865e+17-3.19015e+17im  │ 1.3e-25   │
│ (4, 1//2)  │ -2.03572e+18-1.59162e+17im │ 1.2e-25   │
│ (4, -1//2) │ -5.54884e+17+3.3561e+17im  │ 1.3e

# Polynomials

In [3]:
import BootstrapVirasoro.LoopModels: compute_reference, ρ_residue

wrs(r, s, β) = 2cospi(r * β^2 - s)

βs = [1 / (big"0.8" + big"0.2" * im) + (i - 1) // 10 for i in 1:10]
cs = [CC(β=β) for β in βs]
DGs = [DoubleGamma(c.β) for c in cs]
inds = [(1, 0) for β in βs]
Ps = [[(big"0.2" + big"0.2" * im + i // 100 * im) * cs[1].β / cs[j].β for i in 1:10] for j = eachindex(cs)] # diag field identical in each channel
blocks = [[precompute_blocks(inds[j], c=c, P=P, parity=1, precision=15) for (i, P) = enumerate(Ps[j])] for (j, c) in enumerate(cs)]
sols = [[solve(blocks[j][i], Sig(0, 0, 0), show=false, rels=:stu) for i = eachindex(Ps[1])] for j = eachindex(cs)];

In [110]:
function compute_times_ref_minus_res(r, s)
    vals = [[sols[j][i].str_cst.s.constants[Field(c, r=r, s=s)] for i = eachindex(Ps[1])] for (j, c) = enumerate(cs)]
    Threads.@threads for j = eachindex(cs)
        c = cs[j]
        DG = DGs[j]
        V = Field(c, r=r, s=s)
        n = -2cospi(c.β^2)
        Threads.@threads for i = eachindex(Ps[j])
            P = Ps[j][i]
            co = sols[j][i].correlation
            diag_field = Field(c, P=P)
            w = 2cospi(2c.β * P)
            DP = compute_reference(co, diag_field, :s, DG)
            DV = compute_reference(co, V, :s, DG)
            vals[j][i] *= DP / DV
            vals[j][i] -= ρ_residue(V, co.fields...) / (w - wrs(V.r, V.s, c.β))
            if r == 1
                vals[j][i] /= n+2
            end
        end
    end
    return [((2cospi(2c.β * Ps[j][i]), -2cospi(c.β^2)), vals[j][i]) for i = eachindex(Ps[1]) for (j, c) = enumerate(cs)]
end

function fit_polynomial(data, varnames, degs)
    if varnames === (:w, :n) || varnames === (:n, :w)
        to_fit = data
    elseif varnames === (:w,)
        to_fit = [((d[1][1],), d[2]) for d in data[1:length(cs):end]]
    elseif varnames === (:n,)
        to_fit = [((d[1][2],), d[2]) for d in data[1:length(cs)]]
    else
        error("Unsupported varnames")
    end
    fit!(Polyfit(varnames, degs), to_fit)
end;

In [111]:
# only run to reset the data dictionary! a bit expensive to recompute
data = Dict()

Dict{Any, Any}()

In [112]:
data[1//2, 0] = compute_times_ref_minus_res(1//2, 0);

In [113]:
for deg in 0:2
    display(fit_polynomial(data[1//2, 0], (:w,), (deg,)))
end

Polyfit((:w,), (0,), 1, [(0,)], Complex{BigFloat}[1.0000000000300208527686551385212321871 - 2.3339913904804421659565305033485048603e-11im])

Polyfit((:w,), (1,), 2, [(0,), (1,)], Complex{BigFloat}[0.99999999991009258327241801862841181687 + 8.0869052851293699226146776603337941514e-11im, 1.0157616159332945139661669382992489932e-11 + 3.3915690497087898290189835450274219189e-11im])

Polyfit((:w,), (2,), 3, [(0,), (1,), (2,)], Complex{BigFloat}[1.0000000001014560961592932820166582277 - 1.2463651932609033686218188210580514944e-10im, -4.0766587077147251016552283289458170199e-11 - 8.3924084943624350279436642277468188409e-11im, -1.3944090673357723569865261943741332694e-11 - 3.300958310853684709087394534114871589e-13im])

In [114]:
data[1, 0] = compute_times_ref_minus_res(1, 0);

In [115]:
for deg in 0:5
    display(fit_polynomial(data[1, 0], (:w,), (deg,)))
end

Polyfit((:w,), (0,), 1, [(0,)], Complex{BigFloat}[-0.71564556237738049580483662168826813876 + 0.10035460300438255637539439593086448441im])

Polyfit((:w,), (1,), 2, [(0,), (1,)], Complex{BigFloat}[0.23145520683797710464221139723696496308 + 0.13226275461312555629094669593159456302im, 0.093381832708393688002757303264032945778 - 0.18940040667423732733003479486433645715im])

Polyfit((:w,), (2,), 3, [(0,), (1,), (2,)], Complex{BigFloat}[-0.34536447112021845073937577031097677699 - 0.0071761623539946763839104883314152358406im, -0.070863952199486994144898136347477045041 + 0.026522119195648068075946127338013824209im, 0.013844423293741422461016095419832193565 + 0.026023042365220442873998653482119854928im])

Polyfit((:w,), (3,), 4, [(0,), (1,), (2,), (3,)], Complex{BigFloat}[0.12976725362885309363338347685164764325 + 0.4059006378883364161190924803564612188im, 0.32114425790841404061293475804032892024 - 0.16032502562774399796198907825091688652im, 0.011631600258076454926858220075868658592 - 0.070716362664750303638083019482794826016im, -0.0064020171150341643705098496328180249945 - 0.0027797569366004715793704351536660798492im])

Polyfit((:w,), (4,), 5, [(0,), (1,), (2,), (3,), (4,)], Complex{BigFloat}[-0.11345339348026148339008882847366765593 - 0.20167687305192923141076291490939007723im, -0.28234125881467879077092672827606721306 - 0.17741882322139725821242518211794846951im, -0.076403664424932856652866563077156662703 + 0.11363364795159604074555047849766293004im, 0.012977002527917330323716741039947822182 + 0.020218088527480164931525805525181391312im, 0.0015687391307180156904372032525106108144 - 0.00043049820811008306684806695864055131944im])

Polyfit((:w,), (5,), 6, [(0,), (1,), (2,), (3,), (4,), (5,)], Complex{BigFloat}[-0.1725924503249024635113087948852048258 + 0.45882427588350674605996121605867973205im, 0.39134043683667103588546549765595228177 + 0.18848172564773115849765194080684176208im, 0.19926281758593205821962434308524630455 - 0.099811353232674546019208545292347599796im, -0.005442830949951139095120894114140292199 - 0.055561365820080908419435898084180631453im, -0.00686455300259575076797573193918554385 - 0.0020331294635603615456898187680862862251im, -0.00021612195334940621829219448632065730221 + 0.00030302094965312828966054703746180593281im])

In [190]:
bs = precompute_blocks((1, 0), parity=1, precision=30);
sys = solve(bs, Sig(0, 0, 0));

┌─────────────────┬──────────────────────┬───────────┐
│ Field           │ Structure constant   │ Rel. err. │
├─────────────────┼──────────────────────┼───────────┤
│ P = (0.4+0.1im) │ 1.0+0.0im            │ 0         │
│ (1//2, 0)       │ -1.85579+1.04513im   │ 1.7e-33   │
│ (1, 0)          │ -0.691991+0.509808im │ 3.3e-31   │
│ (1, 1)          │ 2.22426-1.63557im    │ 4.8e-32   │
│ (3//2, 0)       │ -1.75498+1.34764im   │ 3e-28     │
│ (3//2, 2//3)    │ 2.38055-1.76623im    │ 1.2e-28   │
│ (2, 0)          │ -3.11385+2.42417im   │ 1.7e-23   │
│ (2, 1//2)       │ -1.9265+1.44319im    │ 2.3e-23   │
│ (2, 1)          │ 2.06139-1.66833im    │ 1.9e-23   │
│ (5//2, 0)       │ -5.24947+4.26608im   │ 3.8e-17   │
│ (5//2, 2//5)    │ 0.944912-0.782701im  │ 2e-16     │
│ (5//2, 4//5)    │ 0.940418-0.783574im  │ 1.9e-16   │
│ (3, 0)          │ 1.06836-0.755555im   │ 1.3e-08   │
│ (3, 1//3)       │ -1.44922+1.20726im   │ 8.6e-09   │
│ (3, 2//3)       │ 2.67629-2.19646im    │ 4.6e-09   │
│ (3, 1)  

In [191]:
co = sys.correlation
c = co.c
V1 = co.fields[1]
n = -2cospi(c.β^2)
DG = DoubleGamma(c.β)
P = big"0.4" + big"0.1" * im
diag_field = Field(c, P=P)
w = 2cospi(2c.β * P)
r = 1
s = 0
V = Field(c, r=r, s=s)
sys.str_cst.s.constants[V] /
(compute_reference(co, V, :s, DG) / compute_reference(co, diag_field, :s, DG) *
((w + 1 - 2 / (w + n))))

0.99999999999999999998903385961691948239 + 7.7987817632529195755533605494111669678e-21im

In [199]:
r = 3//2
s = 0
V = Field(c, r=r, s=s)
sys.str_cst.s.constants[V] /
(compute_reference(co, V, :s, DG) / compute_reference(co, diag_field, :s, DG) *
(-2 / 3 * ((n+1) * w^2 + n - 2)) / (2*(n+1)/(n-1)^2))

0.0033545050249853068356820746807407031552 + 0.012652628732974413489940083306350526067im

In [167]:
sys.str_cst.s.constants[V] / (3.4997109619837552794971206100000169484 - 2.7027618168568767035607529037691153922im)

-0.50040006657439859992940719365148842339 - 0.001378668859163222165956371417972987067im

In [119]:
sys.str_cst.s.constants[Field(c, r=1, s=0)] * 
compute_reference(co, diag_field, :s, DG) / compute_reference(co, Field(c, r=r, s=s), :s, DG) -
ρ_residue(V, V1) / (w+n) / (n+2)

0.45254719741991413478165861088351547813 - 0.049629584229456136013914336971859637735im

In [143]:
sys.str_cst.s.constants[V] * 
compute_reference(co, diag_field, :s, DG) / compute_reference(co, V, :s, DG)# -
# ρ_residue(V, V1) / prod(
#     sinpi(c.β^2 * j + s)
#     for j in 1-r:r-1 if j != 0
# ) / 2^7 / (w - wrs(2, 0, c.β))

-5935.1712243057739686562226759878885402 - 3861.4944239328445410459169080362451751im

In [97]:
wrs(1, 0, c.β)

-22.781359522846832243243821565023004126 + 73.898565163616871378137947722903776072im

In [ ]:
ρ_residue()